In [2]:
import os
import pandas as pd

workdir = os.path.abspath('./')
print(workdir)

/Users/dongjiacheng/Desktop/code/lab/YM


In [7]:
# pd.读取excel
# df_sc = pd.read_excel('./autophagy gene list-yeast-total 268 gene.xlsx')
df_sc = pd.read_excel('./Nc_蛋白分泌.xlsx',sheet_name='Nc')
df_sc.head()

,Gene,Ano_seq
0,NCU10927,MDITKFVVSHRENALLYGDYATYRAQLSRKLLNSRKKLNIATKSRG...
1,NCU01455,MSDSAIAALNSLLRGTSIDDHEEALKLANSAIKAAKGNPADLTTAQ...
2,NCU03485,MPSHPRVEEVEDSDLEMSDPSEGDIDDFAESDILVQRDAPPAYASK...
3,NCU01146,MVNFRWCSLALLALRAVSVFAQEETFGDDSTAAPVSQPELKADIET...
4,NCU09696,MVLQDLGRRINAAVSDLTRAPNLDEKAFDGMLKEICSALLEADVNV...


In [8]:
"""
将df_sc中的数据，保存到一个txt中，要求格式如下：
>Protein_id + Gene + Symbol
Amin_seq
.........
"""
# 将df_sc中的数据，保存到一个txt中
with open('blast_input_Nc.txt', 'w') as f:
    for i in range(len(df_sc)):
        gene = df_sc['Gene'][i]
        amino_seq = df_sc['Ano_seq'][i]
        
        # 检查是否为NaN值，如果是NaN则将其转换为空字符串
        if pd.isna(gene):
            gene = ""
        if pd.isna(amino_seq):
            amino_seq = ""
        
        f.write('>' + gene + '\n')
        f.write(amino_seq + '\n')


In [9]:
# 在jupyter中添加环境变量
os.environ['PATH'] += os.pathsep + '/Users/dongjiacheng/Desktop/code/mtd/tools/blast/bin'

In [7]:
# os.system('blastp -query blast_input_Sc.txt -out blast_output_Sc.txt -db /Users/dongjiacheng/Desktop/code/mtd/tools/blast/db_prot/Mt-Nc -outfmt 6 -evalue 1e-6')

0

In [11]:
os.system('blastp -query blast_input_Nc.txt -out blast_output_Nc.txt -db /Users/dongjiacheng/Desktop/code/mtd/tools/blast/db_prot/Myceliophthora_thermophila_ATCC_42464 -outfmt 6 -evalue 1e-6')

0

In [12]:
# 读取blast_output_Sc.txt,
df_blast = pd.read_csv('blast_output_Nc.txt', sep='\t', header=None).copy()
df_blast.columns = ['query_id', 'subject_id', '%_Identity', 'alignment_length ', 'mismatches ', 'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e-value', 'bit_score']

df_blast.head()

,query_id,subject_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score
0,NCU10927,XP_003660447.1,60.602,665,220,9,1,630,1,658,0.000000e+00,715.0
1,NCU01455,XP_003661609.1,65.967,667,217,5,1,659,1,665,0.000000e+00,858.0
2,NCU03485,XP_003666635.1,56.000,325,92,5,3,315,2,287,4.600000e-100,297.0
3,NCU01146,XP_003661433.1,67.266,278,80,3,1,267,1,278,5.030000e-131,370.0
4,NCU09696,XP_003661045.1,95.464,463,21,0,1,463,1,463,0.000000e+00,914.0


In [13]:
df_blast['e-value'] = df_blast['e-value'].apply(lambda x: '{:.10f}'.format(x))

# df_blast改名为Protein_id
df_blast.rename(columns={'subject_id': 'Protein_id'}, inplace=True)

# Protein_id不保留小数点及后面的数字
df_blast['Protein_id'] = df_blast['Protein_id'].apply(lambda x: x.split('.')[0])

df_blast.head()

,query_id,Protein_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score
0,NCU10927,XP_003660447,60.602,665,220,9,1,630,1,658,0.0000000000,715.0
1,NCU01455,XP_003661609,65.967,667,217,5,1,659,1,665,0.0000000000,858.0
2,NCU03485,XP_003666635,56.000,325,92,5,3,315,2,287,0.0000000000,297.0
3,NCU01146,XP_003661433,67.266,278,80,3,1,267,1,278,0.0000000000,370.0
4,NCU09696,XP_003661045,95.464,463,21,0,1,463,1,463,0.0000000000,914.0


#### 添加匹配蛋白的注释信息

In [14]:
# 读取Mt-Gene_information.tsv
df_mt = pd.read_csv('./Mt-Gene_information.tsv', sep='\t').copy()
df_mt = df_mt[['Protein_id','Gene','Description','Pathway Name','Pathway ID','GO ID','PFAM','Amino_acid_sequence']]

df_mt.head()

,Protein_id,Gene,Description,Pathway Name,Pathway ID,GO ID,PFAM,Amino_acid_sequence
0,XP_003660026,MYCTH_100011,2OG-Fe(II) oxygenase superfamily,NaN,NaN,NaN,2OG-FeII_Oxy_2,MSNITNKVISQTLSPVDVPPKTFTSSDTTKSRLAAQRLRIATKRAA...
1,XP_003662402,MYCTH_100068,Belongs to the glycosyl hydrolase 11 (cellulas...,NaN,NaN,"GO:0005576,GO:0005975,GO:0030248,GO:0004553,GO...","CBM_1,Glyco_hydro_11",MVALSSLLVAASAAAVAVAAPSEALQKRQTLTSSQTGFHDGFYYSF...
2,XP_003662373,MYCTH_100089,"protocatechuate 3,4-dioxygenase activity",NaN,NaN,"GO:0005506,GO:0016702,GO:0055114",Dioxygenase_C,MTLKQLLMSALLAAPALAHPAHKEKVYAHAALPLERKSLAHCAAKF...
3,XP_003662324,MYCTH_100127,YCII-related domain,NaN,NaN,NaN,YCII,MATEAPKKMEWLVVVPDFPGAHEKRIAVRPQHFANLGPVKESGVFQ...
4,XP_003662142,MYCTH_100290,Mis6,NaN,NaN,"GO:0000776,GO:0034508",CENP-I,MPSPEDDDLGGLIGDLEAASKVPAKRRQTNIKSTVEKATALLYDRG...


In [11]:
df_nc = pd.read_csv('./Nc-Gene_information.tsv', sep='\t').copy()
df_nc = df_nc[['Protein_id','Gene','Description','Pathway Name','Pathway ID','GO ID','PFAM','Amino_acid_sequence']]

df_nc.head()

,Protein_id,Gene,Description,Pathway Name,Pathway ID,GO ID,PFAM,Amino_acid_sequence
0,XP_956990,NCU00003,NaN,NaN,NaN,NaN,NaN,MAKSLSDMVELISSPTAVKSLDSDVSPRTQNSSKPKDYFSPQPRAH...
1,XP_956991,NCU00004,Universal stress protein family,NaN,NaN,NaN,Usp,MSSSLTKSSNSAPRFRQHVGFDNLPAGDSTKNNTSSFTLHVKHQGY...
2,XP_956992,NCU00005,Abscisic acid G-protein coupled receptor,NaN,NaN,GO:0016020,"ABA_GPCR,GPHR_N",MSCGVDSCGAPIPTSPIGTLFSLSPFIATFAIVSSVVAGKVFPKLS...
3,XP_011393719,NCU00006,Src homology 3 domains(BUD14),NaN,NaN,GO:0005515,"RA,SH3_1",MTRPEIIRADTIDLQSRGAHSTQGFHSSTSNGPLAPHQAETLREVA...
4,XP_956994,NCU00007,PalH/RIM21(RIM21),NaN,NaN,NaN,PalH,MEPRQLFSDPTADPISTAGAASNALSCASFNLPEGGILQLPNGEII...


In [15]:
"""
如果df_blast中的Protein_id与df_mt中Protein_id是一样的
那么就将df_mt中的Gene、Description、Pathway Name、Pathway ID、GO ID、PFAM、Amino_acid_sequence添加到df_blast中
"""

df_1 = pd.merge(df_blast, df_mt, on='Protein_id', how='left')
df_1


,query_id,Protein_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,e-value,bit_score,Gene,Description,Pathway Name,Pathway ID,GO ID,PFAM,Amino_acid_sequence
0,NCU10927,XP_003660447,60.602,665,220,9,1,630,1,658,0.0000000000,715.0,MYCTH_2298795,Signal-recognition-particle assembly has a cru...,Mt-Protein export,mtm03060,"GO:0005047,GO:0006614,GO:0008312,GO:0005786,GO...",SRP68,MDITKTAVSLREKALLYGDYSTYWSQLSGKLLNSRKKLNIATKHRG...
1,NCU01455,XP_003661609,65.967,667,217,5,1,659,1,665,0.0000000000,858.0,MYCTH_2301184,Signal-recognition-particle assembly has a cru...,Mt-Protein export,mtm03060,"GO:0006614,GO:0005515,GO:0006614,GO:0008312,GO...","SRP72,SRP_TPR_like",MGDTAVAALNALLRGASIDDHAEALDLASAALRSAKGRGAQLVAAQ...
2,NCU03485,XP_003666635,56.000,325,92,5,3,315,2,287,0.0000000000,297.0,MYCTH_2311498,SRP19 protein(SEC65),Mt-Protein export,mtm03060,"GO:0006614,GO:0008312,GO:0048500",SRP19,MSHPRVEEVSDSDVDMSDPSEGDIDELIESDIMRRVDSKPQSSRPQ...
3,NCU01146,XP_003661433,67.266,278,80,3,1,267,1,278,0.0000000000,370.0,MYCTH_2300802,signal sequence receptor alpha chain(IRC22),NaN,NaN,NaN,TRAP_alpha,MVNFKWSSLALLALRVATAFAADDADAEPVTSVPVEEPELKADIET...
4,NCU09696,XP_003661045,95.464,463,21,0,1,463,1,463,0.0000000000,914.0,MYCTH_2299988,Binds to the signal sequence of presecretory p...,Mt-Protein export,mtm03060,"GO:0006614,GO:0008312,GO:0003924,GO:0048500,GO...","SRP54,SRP54_N,SRP_SPB",MVLQDLGRRINAAVSDLTRAPNLDEKAFDSMLKQICSALLEADVNV...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,NCU03292,XP_003664005,33.714,175,100,5,637,810,996,1155,0.0000000000,82.0,MYCTH_102797,Heavy-metal-associated domain,NaN,NaN,"GO:0030001,GO:0046872,GO:0000166,GO:0016021,GO...","E1-E2_ATPase,HMA,Hydrolase",MAPPSSRRPVAAGPSETVSTSFLLANLHCPTCVSTIKSALHGSCGS...
1660,NCU03292,XP_003663560,21.398,472,290,13,351,769,429,872,0.0000000000,69.3,MYCTH_2305576,Cation-transporting ATPase(SPF1),NaN,NaN,"GO:0000166,GO:0016021,GO:0006812,GO:0016887,GO...","E1-E2_ATPase,Hydrolase",MAPLVDNEQIQSAELLRPLPLYQHAYVWPFAIIWPVFLRYYLTPEL...
1661,NCU03292,XP_003663518,28.481,158,91,2,637,794,916,1051,0.0000000001,63.9,MYCTH_2061857,haloacid dehalogenase-like hydrolase,NaN,NaN,"GO:0030001,GO:0046872,GO:0000166,GO:0016021,GO...","E1-E2_ATPase,HMA,Hydrolase",MACSSGCCGKPASPPPPEQPNSNPASTPDQDDGDSCCDGANTIAGE...
1662,NCU03292,XP_003662050,24.014,583,327,26,297,783,542,1104,0.0000000289,56.6,MYCTH_2302129,Belongs to the cation transport ATPase (P-type...,NaN,NaN,"GO:0016021,GO:0015914,GO:0000287,GO:0005524,GO...","Cation_ATPase,E1-E2_ATPase,Hydrolase,PhoLip_AT...",MRSSTTHPPAKPPDSPADHSDTDLELDLEELDPQTVSGSDQPRGRG...


In [13]:
# 如果df_1中的Protein_id与df_nc中Protein_id是一样的
# 那么就将df_nc中的Gene、Description、Pathway Name、Pathway ID、GO ID、PFAM、Amino_acid_sequence添加到df_1中
df_2 = pd.merge(df_1, df_nc, on='Protein_id', how='left')
df_2

,query_id,Protein_id,%_Identity,alignment_length,mismatches,gap_openings,q_start,q_end,s_start,s_end,...,GO ID_x,PFAM_x,Amino_acid_sequence_x,Gene_y,Description_y,Pathway Name_y,Pathway ID_y,GO ID_y,PFAM_y,Amino_acid_sequence_y
0,YDR425W,XP_962863,26.145,524,345,8,122,623,112,615,...,NaN,NaN,NaN,NCU06259,Vps5 C terminal like(ATG20),NaN,NaN,GO:0035091,"PX,Vps5",MWNDEDNNPYGSFERRDSFASSTNPASPTARDYSIDARFPTPQDVR...
1,YDR425W,XP_003663337,32.869,359,212,6,122,472,97,434,...,GO:0035091,"PX,Vps5",MWNDEDNNPYGGSFERRDSFASSANPSSPITHDYPRYDAPNTPSFT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YJL036W,XP_003658349,34.038,426,234,6,28,418,56,469,...,GO:0035091,"PX,Vps5",MAVLDQDNFSNISWHSEQNPDAAGPSTSASSHDRSHEARTGDGRHV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YJL036W,XP_965318,33.333,423,238,6,28,418,69,479,...,NaN,NaN,NaN,NCU03218,Vps5 C terminal like(SNX4),Nc-Endocytosis,ncr04144,GO:0035091,"PX,Vps5",MAVIDQDNFSNISWHSEQNAESAASTAQVHHESNSSPEYARSGPDD...
4,YJL036W,XP_011393870,23.810,252,146,8,75,299,444,676,...,NaN,NaN,NaN,NCU05715,"PhoX homologous domain, present in p47phox and...",NaN,NaN,"GO:0035091,GO:0035091,GO:0042147",PX,MSLFGSSPPNDGSAALNPAKTANSSRSTLFDNEAPTTRSGSALFAD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6784,YPR080W,XP_958508,22.814,469,266,19,10,439,199,610,...,NaN,NaN,NaN,NCU09513,Elongation factor Tu domain 2,NaN,NaN,"GO:0003924,GO:0005525,GO:0005525","GTP_EFTU,GTP_EFTU_D2,GTP_EFTU_D3",MASKQAPHERRKGEAALSDFAEYVERQQALRYPNKAAQSQTATTSA...
6785,YPR080W,XP_003662584,22.979,470,264,19,10,439,191,602,...,"GO:0005525,GO:0003924,GO:0005525","GTP_EFTU,GTP_EFTU_D2,GTP_EFTU_D3",MTSKSAKQSSHERRKGEAALSDFAEYVEKQQALRYPSSKQVAATAA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6786,YPR080W,XP_958122,26.354,277,143,9,9,279,123,344,...,NaN,NaN,NaN,NCU05927,Promotes mitochondrial protein synthesis. May ...,NaN,NaN,"GO:0003924,GO:0005525,GO:0005525","EFG_C,GTP_EFTU,GTP_EFTU_D2,LepA_C",MRGALCRPDVLMRPCLRPCARLPRLPPSRRLPFSITTSCQQLPRTS...
6787,YPR080W,XP_962160,27.094,203,106,8,9,200,58,229,...,NaN,NaN,NaN,NCU07021,P-loop containing nucleoside triphosphate hydr...,NaN,NaN,"GO:0003924,GO:0005525,GO:0005525","EFG_C,EFG_II,GTP_EFTU,GTP_EFTU_D2",MLLAAPLRGCRHGHRAFLPLSLPTVVKSTSPLAATRFPHFKRFYAD...


In [16]:
# 保存df_blast为excel   
# df_2.to_excel('比对结果.xlsx', index=False)
df_1.to_excel('蛋白分泌比对结果.xlsx', index=False)